In [95]:
import sys
sys.path.append('..')
from db1 import *
import pandas as pd

R = PolynomialRing(ZZ, 't')
t = R.gen()

def is_paving(M):
    n = M.size()
    r = M.rank()
    return (len(M.independent_r_sets(r-1)) == binomial(n, r-1))

def q_kl(k, h):
    return kazhdan_lusztig_inverse_uniform(k, h+1) - kazhdan_lusztig_inverse_uniform(k-1, h)

def kl_inverse_fast(M):
    if M.loops(): return R(0)
    k, n = M.rank(), M.size()
    if k == n or k == 0: return R(1)
    if not M.is_connected():
        ans = R(1)
        CC = M.components()
        for N in CC:
            res = M.delete(M.groundset() - N)
            ans = ans * kl_inverse_fast(res)
        return ans

    if is_paving(M):
        return kl_inverse_paving(M)
    if is_paving(M.dual()):
        return kl_inverse_copaving(M)
    """
    if n <= 8 and M.is_connected():
        for i in range(len(mat[n][k])):
            if mat[n][k][i].is_isomorphic(M):
                return ikl[n][k][i]
    """
    LF = M.lattice_of_flats()
    ans = R(0)
    for F in LF:
        if len(F) != n:
            Res = M.delete(M.groundset() - F)
            Con = M.contract(F)
            chi = characteristic_polynomial(Con)(1/t) * t**(Con.rank())
            PPP = kl_inverse_fast(Res)(t) * (-1)**(Res.rank())
            ans = ans + chi * PPP
    assert (t**k * ans(1/t)).numerator() == -ans(t)
    ans = ans.numerator() * (-1)**(k+1)
    return ans.truncate((k+1)//2)

def kazhdan_lusztig_inverse_uniform(k, n):
    if k == n:
        return R(1)
    d = k
    m = n - d
    ans = 0
    for j in range((d-1)//2 + 1):
        ans = ans + m * (d-2*j)/((m+j) * (m+d-j)) * binomial(d, j) * t**j
    return ans * binomial(m+d, d)

def kl_inverse_paving(M):
    assert is_paving(M)
    n = M.size()
    k = M.rank()
    ans = kazhdan_lusztig_inverse_uniform(k, n)
    for H in M.hyperplanes():
        h = len(H)
        if h >= k:
            ans = ans - q_kl(k, h)
    return ans

def kl_inverse_copaving(M):
    assert is_paving(M.dual())
    n = M.size()
    k = M.rank()
    ans = kazhdan_lusztig_inverse_uniform(k, n)
    for H in M.dual().hyperplanes():
        h = len(H)
        if h >= n-k:
            ans = ans - kli_vtilde_dual(n-k, h, n) + kazhdan_lusztig_inverse_uniform(h-n+k+1, h) * kazhdan_lusztig_inverse_uniform(n-h-1, n-h)
    return ans

def kli_vtilde_dual(k, h, n):
    return helper1(n-k, h, n)

def helper1(k, h, n):
    c = n - h
    ans1 = kazhdan_lusztig_inverse_uniform(k, n)
    ans2 = helper2(c, k, n)
    ans3 = kazhdan_lusztig_inverse_uniform(k-c+1, h) * kazhdan_lusztig_inverse_uniform(c-1, c)
    return ans1 - ans2 + ans3

def helper2(c, k, n):
    h = n - c
    ans = 0
    for j in range(k-c+1):
        ans = ans + binomial(n-c, j) * (-1)**(c-1+j) * kazhdan_lusztig_inverse_uniform(c-1, c) * t**(k-c-j+1) * chuly(k-c-j+1, n-c-j)(1/t)
    for i in range(c-1):
        for j in range(k-i):
            ans = ans + binomial(c, i) * binomial(n-c, j) * (-1)**(i+j) * t**(k-i-j) * helper4(c, k, n, i, j)(1/t)
    ans = ans.numerator().truncate((k-1)//2 + 1)
    if ans[0] < 0:
        ans = -ans
    return ans

def helper3(c, k, n):
    ans = 0
    for j in range(k-c+1):
        ans = ans + binomial(n-c, j) * kazhdan_lusztig_uniform_matroid(c-1, c) * (-1)**(k-c-j+1) * kazhdan_lusztig_inverse_uniform(k-c-j+1, n-c-j)
    for i in range(c-1):
        for j in range(k-i):
            ans = ans + binomial(c, i) * binomial(n-c, j) * (-1)**(k-i-j) * helper2(c-i, k-i-j, n-i-j)
    return -ans

def helper4(c, k, n, i, j):
    ans = 0
    for l in range(c-i-1):
        ans = ans + (-1)**l * (t-1)**(max(n-i-j-l-1, 0))
    for u in range(n-k-1):
        ans = doit_once(ans)
    return ans

def chuly(a, b):
    ans = (t-1)**b
    for i in range(b-a):
        ans = doit_once(ans)
    return ans

def doit_once(p):
    p = p // t**2
    p = p * t
    p = p - p(1)
    return p

def lorenzo(k, h, n):
    c = n - h
    ans1 = kazhdan_lusztig_uniform_matroid(k, n) + kazhdan_lusztig_uniform_matroid(k-c+1, h) * kazhdan_lusztig_uniform_matroid(c-1, c)
    ans2 = helper3(c, k, n)
    return ans1 - ans2

In [96]:
def parallel_connection(m, n):
    G = graphs.CycleGraph(m + n - 2)
    G.add_edge(0, m-1)
    edge_e = frozenset({(0, m - 1)})
    return G, edge_e

def kl(M):
    return M.lattice_of_flats().kazhdan_lusztig_polynomial().factor()(t)

def set_division(M, e):
    L = M.lattice_of_flats()
    divs = {'A': set(), 'B': set(), 'C': set(), 'D': set()}
    for x in L[1:]:
        if set(e).issubset(set(x)):
            if frozenset(set(x) - set(e)) in L[1:]:
                divs['A'].add(x)
            else:
                divs['D'].add(x)
        else:
            if frozenset(set(x) | set(e)) in L:
                divs['C'].add(x)
            else:
                divs['B'].add(x)
    return divs

In [97]:
n = 5
G, e = parallel_connection(n, n)
M = Matroid(G)

flats = M.lattice_of_flats().list()
gset = M.groundset()
labels = {element: idx for idx, element in enumerate(gset)}
relabel_flats = {flat: frozenset({labels[element] for element in flat}) for flat in flats}

In [98]:
def q_tau(matroid, S, e):
    q_tau = [] 
    for F in S:
        modded_matroid = matroid.delete(matroid.groundset() - F)
        if modded_matroid.rank() % 2 == 0:
            q_tau.append(0)
        else:
            q_tau.append(kl_inverse_fast(modded_matroid).leading_coefficient())
    return q_tau

def the_set_S(flats, e):
    return set(F for F in flats if e.isdisjoint(F) and F.union(e) in flats)

def delform_rhs(matroid, e):
    contraction = matroid.contract(e).simplify()
    return (t + 1) * kl_inverse_fast(contraction)

def sum_S(matroid, e):
    flats = list(matroid.lattice_of_flats())
    set_S = list(the_set_S(flats, e))
    tau_values = q_tau(matroid, set_S, e)

    sum_S = sum(
        x * t ** (matroid.rank(y)//2) * kl_inverse_fast(matroid.contract(y))
        for x, y in zip(tau_values, set_S)
    )
    return sum_S

In [99]:
divs = set_division(M, e)
divs_deletion = {x: set(frozenset(set(y) - set(e)) - {frozenset()} for y in divs[x]) for x in divs}

for key, value in divs.items():
    print(f"{key}: {[relabel_flats[v] for v in value]}")

divs_contraction = {x: set(frozenset(set(y) - set(e)) for y in divs[x] if set(e).issubset(set(y))) - {frozenset()} for x in divs}
e

A: [frozenset({3, 4, 5, 6, 8}), frozenset({0, 1, 3, 5, 6}), frozenset({1, 3, 4, 5, 8}), frozenset({1, 3, 4, 7}), frozenset({0, 3, 6}), frozenset({8, 3, 4}), frozenset({0, 1, 3}), frozenset({2, 3}), frozenset({0, 1, 3, 5}), frozenset({8, 2, 3, 6}), frozenset({2, 3, 6}), frozenset({0, 2, 3, 6, 7}), frozenset({2, 3, 4, 7, 8}), frozenset({0, 8, 3, 4}), frozenset({1, 3, 5, 6}), frozenset({1, 3, 4, 5, 7}), frozenset({2, 3, 5}), frozenset({1, 2, 3, 4, 6}), frozenset({0, 1, 2, 3, 8}), frozenset({1, 3, 4, 5}), frozenset({0, 1, 3, 4}), frozenset({1, 3}), frozenset({3, 5, 6}), frozenset({0, 8, 3, 7}), frozenset({3, 4, 6, 7}), frozenset({0, 3, 7}), frozenset({8, 3, 5, 7}), frozenset({2, 3, 5, 6}), frozenset({0, 2, 3, 7}), frozenset({0, 1, 2, 3, 6}), frozenset({1, 2, 3, 4, 8}), frozenset({1, 2, 3}), frozenset({0, 3, 5, 6}), frozenset({8, 3, 4, 5}), frozenset({2, 3, 4, 6, 8}), frozenset({8, 3, 7}), frozenset({8, 3, 6}), frozenset({0, 3, 4}), frozenset({0, 1, 3, 4, 7}), frozenset({3, 4, 7}), frozense

frozenset({(0, 4)})

In [100]:
for key, value in divs_contraction.items():
    print(f"{key}: {value}")

A: {frozenset({(2, 3), (0, 7), (1, 2)}), frozenset({(0, 1), (0, 7)}), frozenset({(0, 1), (0, 7), (4, 5), (1, 2)}), frozenset({(0, 1), (0, 7), (1, 2), (6, 7)}), frozenset({(0, 7), (1, 2), (3, 4), (5, 6)}), frozenset({(2, 3), (4, 5), (1, 2), (5, 6)}), frozenset({(2, 3), (0, 7)}), frozenset({(2, 3), (0, 7), (4, 5), (1, 2)}), frozenset({(0, 7), (4, 5), (3, 4)}), frozenset({(6, 7), (5, 6)}), frozenset({(0, 1), (3, 4)}), frozenset({(0, 1), (4, 5), (2, 3)}), frozenset({(0, 1), (4, 5), (2, 3), (5, 6)}), frozenset({(0, 1), (6, 7), (3, 4)}), frozenset({(2, 3), (6, 7), (4, 5), (3, 4)}), frozenset({(0, 1), (4, 5), (3, 4)}), frozenset({(0, 1), (2, 3)}), frozenset({(2, 3), (3, 4)}), frozenset({(6, 7), (1, 2), (5, 6)}), frozenset({(2, 3), (6, 7), (5, 6), (3, 4)}), frozenset({(0, 1), (2, 3), (5, 6)}), frozenset({(0, 7), (1, 2), (6, 7)}), frozenset({(2, 3), (6, 7), (3, 4)}), frozenset({(6, 7), (4, 5)}), frozenset({(6, 7), (1, 2)}), frozenset({(2, 3), (0, 7), (1, 2), (5, 6)}), frozenset({(2, 3), (0, 7),

In [101]:
deletion_flats = M.delete(e).lattice_of_flats().list()

relabel_flats_deletion = {flat: frozenset({labels[element] for element in flat}) for flat in deletion_flats}

for key, value in divs_deletion.items():
    print(f"{key}: {[relabel_flats_deletion[v] for v in value]}")

A: [frozenset({0, 1}), frozenset({0, 1, 2, 7}), frozenset({0, 1, 2, 6}), frozenset({8, 1, 2, 4}), frozenset({1, 5}), frozenset({1, 2, 5, 7}), frozenset({8, 6}), frozenset({0, 4}), frozenset({0, 4, 6}), frozenset({0, 4, 7}), frozenset({4, 5}), frozenset({8, 4, 5, 6}), frozenset({0, 8, 5}), frozenset({1, 2, 6}), frozenset({4, 5, 6}), frozenset({2, 6}), frozenset({8, 1, 2, 5}), frozenset({0, 8, 6}), frozenset({1, 2, 4, 6}), frozenset({0, 1, 2, 8}), frozenset({4}), frozenset({0, 8, 2, 6}), frozenset({4, 7}), frozenset({1, 6}), frozenset({7}), frozenset({8, 1, 2}), frozenset({0, 5, 6}), frozenset({8, 4, 5}), frozenset({0, 2, 6, 7}), frozenset({8, 2, 4, 7}), frozenset({2, 4, 6}), frozenset({0, 8, 2}), frozenset({5, 6}), frozenset({0, 6, 7}), frozenset({8, 4, 7}), frozenset({0, 1, 7}), frozenset({0, 1, 4}), frozenset({0, 1, 5, 7}), frozenset({2, 6, 7}), frozenset({0, 8, 4, 7}), frozenset({1, 2, 4, 7}), frozenset({1, 4}), frozenset({8, 1, 5}), frozenset({2, 5, 7}), frozenset({8, 2}), frozenset

In [102]:
def sum_at_Subset(S, M):
    return sum((-1) ** M.rank(F) * kl(M.delete(M.groundset() - set(F))) * kl_inverse_fast(M.contract(F)) for F in S)

results = {
        'Subset': [],
        'Sum at Subset': [],
        'Sum at Subset Deletion': [],
        'Sum at Subset Contraction': [],
        'difference': []
    }

for x in divs:
    results['Subset'].append(x)
    results['Sum at Subset'].append(sum_at_Subset(divs[x], M))
    results['Sum at Subset Deletion'].append(sum_at_Subset(divs_deletion[x], M.delete(e)))
    results['Sum at Subset Contraction'].append(sum_at_Subset(divs_contraction[x], M.contract(e)))
    results['difference'].append(results['Sum at Subset Deletion'][-1] - results['Sum at Subset'][-1])

results_df = pd.DataFrame(results)
display(results_df)

,Subset,Sum at Subset,Sum at Subset Deletion,Sum at Subset Contraction,difference
0,A,8*t + 8,28*t^2 - 28*t + 8,-8*t - 8,28*t^2 - 36*t
1,B,24*t - 16,24*t - 16,0,0
2,C,4*t^2 - 56*t,28*t^2 - 28*t + 8,0,24*t^2 + 28*t + 8
3,D,-10*t^3 - 36*t^2 - 17*t - 8,-28*t^2 + 4*t + 8,-4*t^2 - 4*t - 1,10*t^3 + 8*t^2 + 21*t + 16


In [104]:
print(kl_inverse_fast(M) - kl_inverse_fast(M.delete(e)) - kl_inverse_fast(M.contract(e)))

-4*t^3 + 9*t


In [105]:
print(f"kl_inverse_fast(M): {kl_inverse_fast(M)}")
print(f"kl_inverse_fast(M.delete(e)): {kl_inverse_fast(M.delete(e))}")
print(f"kl_inverse_fast(M.contract(e)): {kl_inverse_fast(M.contract(e))}")

kl_inverse_fast(M): 10*t^3 + 32*t^2 + 41*t + 16
kl_inverse_fast(M.delete(e)): 14*t^3 + 28*t^2 + 20*t + 7
kl_inverse_fast(M.contract(e)): 4*t^2 + 12*t + 9
